In [1]:
import math
import time
import warnings
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from cleanlab import Datalab
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import (accuracy_score, balanced_accuracy_score,
                             classification_report, confusion_matrix, f1_score,
                             matthews_corrcoef, mean_squared_error)
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [2]:
pd.set_option("display.float_format", lambda x: "%.5f" % x)
pd.set_option("display.max_columns", None)
warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
def train_and_evaluate_models(
    X_train: pd.DataFrame, y_train: pd.Series, num_jobs: int = -1
) -> Tuple[pd.DataFrame, List]:
    """
    Train a variety of models and evaluate them using cross-validation.

    Args:
        X_train (pd.DataFrame): The training data.
        y_train (pd.Series): The training labels.
        num_jobs (int, optional): The number of jobs to run in parallel. Defaults to -1 (use all processors).

    Returns:
        Tuple[pd.DataFrame, List]: A DataFrame of the evaluation results and a list of classification reports.
    """

    # Calculate class weights for balancing
    class_weights = dict(pd.Series(y_train).value_counts(normalize=True))

    # Define the models to be trained
    models = []
    models.append(
        (
            "Calibrated-LSVC",
            CalibratedClassifierCV(
                LinearSVC(random_state=314, class_weight="balanced", dual="auto")
            ),
        )
    )
    models.append(
        (
            "LogisticRegression",
            LogisticRegression(
                random_state=314, n_jobs=num_jobs, class_weight="balanced"
            ),
        )
    )
    models.append(
        (
            "RandomForest",
            RandomForestClassifier(
                random_state=314, n_jobs=num_jobs, class_weight="balanced"
            ),
        )
    )
    models.append(
        (
            "XGBoost",
            XGBClassifier(
                random_state=314, n_jobs=num_jobs, class_weight="balanced", verbosity=0
            ),
        )
    )
    models.append(("MLP", MLPClassifier(random_state=314)))
    models.append(
        (
            "SGD",
            SGDClassifier(random_state=314, n_jobs=num_jobs, class_weight="balanced"),
        )
    )
    models.append(("NaiveBayes", MultinomialNB()))
    models.append(
        ("LinearSVC", LinearSVC(random_state=314, class_weight="balanced", dual="auto"))
    )
    models.append(("KNN", KNeighborsClassifier(n_jobs=num_jobs)))
    models.append(
        (
            "DecisionTree",
            DecisionTreeClassifier(random_state=314, class_weight="balanced"),
        )
    )
    models.append(
        (
            "ExtraTrees",
            ExtraTreesClassifier(
                random_state=314, n_jobs=num_jobs, class_weight="balanced"
            ),
        )
    )
    models.append(
        (
            "CatBoost",
            CatBoostClassifier(
                random_state=314, verbose=0, class_weights=class_weights
            ),
        )
    )

    num_folds = 10
    evaluation_results = []
    classification_reports = []
    stratified_kfold = StratifiedKFold(
        n_splits=num_folds, shuffle=True, random_state=314
    )

    # Train and evaluate each model
    for model_name, model in models:
        start_time = time.time()

        try:
            # Use cross-validation to get out-of-sample predictions
            predictions = cross_val_predict(
                model, X_train, y_train, cv=stratified_kfold, n_jobs=2
            )

        except Exception as e:
            print(f"Error {model_name} - {e}")
            continue

        # Calculate evaluation metrics
        f1 = f1_score(y_train, predictions, average="micro")
        balanced_accuracy = balanced_accuracy_score(y_train, predictions)
        accuracy = accuracy_score(y_train, predictions)
        mcc = matthews_corrcoef(y_train, predictions)
        confusion_mat = confusion_matrix(y_train, predictions)
        class_report = classification_report(y_train, predictions)
        rmse = math.sqrt(mean_squared_error(y_train, predictions))

        elapsed_time = time.time() - start_time
        evaluation_results.append(
            [
                model_name,
                f1,
                balanced_accuracy,
                accuracy,
                mcc,
                rmse,
                elapsed_time,
                confusion_mat,
                class_report,
            ]
        )

        # Print evaluation results
        print(
            f"Name: {model_name} - F1: {f1:.4f} - Balanced Accuracy: {balanced_accuracy:.4f} - Accuracy: {accuracy:.4f} - MCC: {mcc:.4f} - RMSE {rmse:.4f} - Elapsed: {elapsed_time:.2f}s"
        )
        print(class_report)
        print(confusion_mat)
        print("*" * 20, "\n")

    # Create a DataFrame of the evaluation results
    columns = [
        "Model",
        "F1",
        "Balanced Accuracy",
        "Accuracy",
        "MCC",
        "RMSE",
        "Total Time",
        "Confusion Matrix",
        "Classification Report",
    ]
    df_results = pd.DataFrame(evaluation_results, columns=columns)
    df_results["Confusion Matrix"] = df_results["Confusion Matrix"].apply(
        lambda x: str(x)
    )

    return df_results, classification_reports

In [4]:
df = pd.read_parquet("output/data_8_classes_one_hot_encoded.parquet")
df

,idfam,V0409____Quantos_cartoes_de_credito_tem,V0410____Quantas_contas_correntes_tem,ANOS_ESTUDO____Anos_de_estudo_da_pessoa_Variavel_derivada_construida_a_partir_dos_quesitos_referentes_a_educacao,V0205____Quantos_comodos_tem_este_domicilio,V0206____Quantos_comodos_estao_servindo_permanentemente_de_dormitorio_para_os_moradores_deste_domicilio,V02111____Quantos_banheiros_com_chuveiro_e_vaso_sanitario_de_uso_exclusivo_dos_moradores_existem_neste_domicilio_inclusive_os_localizados_no_terreno_ou_na_propriedade,pessoas_na_familia,imc_medio_familia,energia_kcal_medio_familia,anos_estudo_medio_familia,idade_media_familia,RENDA_TOTAL,renda_total_sm,categoria_renda,uid,TIPO_SITUACAO_REG____Situacao_do_Domicilio_Urbano,V0405____Cor_ou_raca_Amarela,V0405____Cor_ou_raca_Branca,V0405____Cor_ou_raca_Indígena,V0405____Cor_ou_raca_Parda,V0405____Cor_ou_raca_Preta,V0405____Cor_ou_raca_Sem declaração,V0406_____tem_plano_ou_segurosaude_Sim,V0407_____teve_algum_tipo_de_rendimento_ou_executou_algum_trabalho_remunerado_ou_nao_no_periodo_de_referencia_de_12_meses_Sim,V0414_____sabe_ler_e_escrever_Sim,V0201____Este_domicilio_e_do_tipo_Apartamento,V0201____Este_domicilio_e_do_tipo_Casa,"V0201____Este_domicilio_e_do_tipo_Habitação em casa de cômodos, cortiço ou cabeça de porco",V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Alvenaria com revestimento / Taipa com revestimento,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Alvenaria sem revestimento,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Madeira apropriada para construção [aparelhada],V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Madeira aproveitada,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Outro material,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Taipa sem revestimento,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Madeira apropriada para construção [aparelhada],V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Outro material,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Somente laje de concreto,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Telha com laje de concreto,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Telha sem laje de concreto,"V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Zinco, alumínio ou chapa metálica","V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Cerâmica, lajota ou pedra",V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Cimento,V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Madeira apropriada para construção [aparelhada],V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Outro Material,V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Terra,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Fonte ou nascente,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Outra forma,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Poço profundo ou artesiano,"V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Poço raso, freático ou cacimba",V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Rede geral de distribuição,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Água da chuva armazenada,V0209____A_agua_utilizada_neste_domicilio_chega_Canalizada em pelo menos um cômodo,V0209____A_agua_utilizada_neste_domicilio_chega_Canalizada só na propriedade ou terreno,V0209____A_agua_utilizada_neste_domicilio_chega_Não canalizada,V0212____De_que_forma_e_feito_o_escoado

In [5]:
for col in df.columns:
    print(col)

idfam
V0409____Quantos_cartoes_de_credito_tem
V0410____Quantas_contas_correntes_tem
ANOS_ESTUDO____Anos_de_estudo_da_pessoa_Variavel_derivada_construida_a_partir_dos_quesitos_referentes_a_educacao
V0205____Quantos_comodos_tem_este_domicilio
V0206____Quantos_comodos_estao_servindo_permanentemente_de_dormitorio_para_os_moradores_deste_domicilio
V02111____Quantos_banheiros_com_chuveiro_e_vaso_sanitario_de_uso_exclusivo_dos_moradores_existem_neste_domicilio_inclusive_os_localizados_no_terreno_ou_na_propriedade
pessoas_na_familia
imc_medio_familia
energia_kcal_medio_familia
anos_estudo_medio_familia
idade_media_familia
RENDA_TOTAL
renda_total_sm
categoria_renda
uid
TIPO_SITUACAO_REG____Situacao_do_Domicilio_Urbano
V0405____Cor_ou_raca_Amarela
V0405____Cor_ou_raca_Branca
V0405____Cor_ou_raca_Indígena
V0405____Cor_ou_raca_Parda
V0405____Cor_ou_raca_Preta
V0405____Cor_ou_raca_Sem declaração
V0406_____tem_plano_ou_segurosaude_Sim
V0407_____teve_algum_tipo_de_rendimento_ou_executou_algum_trabalh

In [6]:
X = df.drop(columns=["categoria_renda", "uid", "idfam", "RENDA_TOTAL", "renda_total_sm"])
y = df["categoria_renda"]

In [7]:
y.nunique()

8

In [8]:
X

,V0409____Quantos_cartoes_de_credito_tem,V0410____Quantas_contas_correntes_tem,ANOS_ESTUDO____Anos_de_estudo_da_pessoa_Variavel_derivada_construida_a_partir_dos_quesitos_referentes_a_educacao,V0205____Quantos_comodos_tem_este_domicilio,V0206____Quantos_comodos_estao_servindo_permanentemente_de_dormitorio_para_os_moradores_deste_domicilio,V02111____Quantos_banheiros_com_chuveiro_e_vaso_sanitario_de_uso_exclusivo_dos_moradores_existem_neste_domicilio_inclusive_os_localizados_no_terreno_ou_na_propriedade,pessoas_na_familia,imc_medio_familia,energia_kcal_medio_familia,anos_estudo_medio_familia,idade_media_familia,TIPO_SITUACAO_REG____Situacao_do_Domicilio_Urbano,V0405____Cor_ou_raca_Amarela,V0405____Cor_ou_raca_Branca,V0405____Cor_ou_raca_Indígena,V0405____Cor_ou_raca_Parda,V0405____Cor_ou_raca_Preta,V0405____Cor_ou_raca_Sem declaração,V0406_____tem_plano_ou_segurosaude_Sim,V0407_____teve_algum_tipo_de_rendimento_ou_executou_algum_trabalho_remunerado_ou_nao_no_periodo_de_referencia_de_12_meses_Sim,V0414_____sabe_ler_e_escrever_Sim,V0201____Este_domicilio_e_do_tipo_Apartamento,V0201____Este_domicilio_e_do_tipo_Casa,"V0201____Este_domicilio_e_do_tipo_Habitação em casa de cômodos, cortiço ou cabeça de porco",V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Alvenaria com revestimento / Taipa com revestimento,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Alvenaria sem revestimento,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Madeira apropriada para construção [aparelhada],V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Madeira aproveitada,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Outro material,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Taipa sem revestimento,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Madeira apropriada para construção [aparelhada],V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Outro material,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Somente laje de concreto,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Telha com laje de concreto,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Telha sem laje de concreto,"V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Zinco, alumínio ou chapa metálica","V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Cerâmica, lajota ou pedra",V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Cimento,V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Madeira apropriada para construção [aparelhada],V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Outro Material,V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Terra,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Fonte ou nascente,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Outra forma,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Poço profundo ou artesiano,"V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Poço raso, freático ou cacimba",V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Rede geral de distribuição,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Água da chuva armazenada,V0209____A_agua_utilizada_neste_domicilio_chega_Canalizada em pelo menos um cômodo,V0209____A_agua_utilizada_neste_domicilio_chega_Canalizada só na propriedade ou terreno,V0209____A_agua_utilizada_neste_domicilio_chega_Não canalizada,V0212____De_que_forma_e_feito_o_escoadouro_destes_banheiros_sanitarios_ou_buracos_para_dejec

In [9]:
X = X.values
y = y.values

In [10]:
# Treinando os modelos
df_results, creports = train_and_evaluate_models(X, y)

/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Name: Calibrated-LSVC - F1: 0.2476 - Balanced Accuracy: 0.1431 - Accuracy: 0.2476 - MCC: 0.0033 - RMSE 1.6817 - Elapsed: 39.20s
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       950
           1       0.00      0.00      0.00      2777
           2       0.23      0.01      0.01      3381
           3       0.23      0.30      0.26      5393
           4       0.25      0.62      0.35      4902
           5       0.40      0.13      0.20      1928
           6       0.41      0.09      0.14       618
           7       0.00      0.00      0.00       195

    accuracy                           0.25     20144
   macro avg       0.19      0.14      0.12     20144
weighted avg       0.21      0.25      0.18     20144

[[   0    0   26  397  526    0    1    0]
 [   0    0   37  935 1804    1    0    0]
 [   0    0   25 1082 2269    5    0    0]
 [   0    0   16 1635 3720   22    0    0]
 [   0    0    3 1765 3019  112    3    0]
 [   0

/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.or

Name: LogisticRegression - F1: 0.1740 - Balanced Accuracy: 0.2620 - Accuracy: 0.1740 - MCC: 0.0635 - RMSE 2.2098 - Elapsed: 10.95s
              precision    recall  f1-score   support

           0       0.09      0.47      0.15       950
           1       0.22      0.18      0.20      2777
           2       0.20      0.06      0.09      3381
           3       0.34      0.16      0.22      5393
           4       0.31      0.18      0.23      4902
           5       0.13      0.19      0.15      1928
           6       0.07      0.19      0.10       618
           7       0.07      0.66      0.13       195

    accuracy                           0.17     20144
   macro avg       0.18      0.26      0.16     20144
weighted avg       0.25      0.17      0.18     20144

[[ 446  156   45   72   89   90   43    9]
 [1032  499  145  341  328  259  113   60]
 [1095  513  198  431  432  405  207  100]
 [1362  780  303  855  792  742  352  207]
 [ 799  306  240  658  887  923  579  510]
 [ 

/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


...

/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Name: ExtraTrees - F1: 0.3716 - Balanced Accuracy: 0.2762 - Accuracy: 0.3716 - MCC: 0.2027 - RMSE 1.2126 - Elapsed: 7.37s
              precision    recall  f1-score   support

           0       0.35      0.11      0.17       950
           1       0.35      0.31      0.33      2777
           2       0.28      0.19      0.23      3381
           3       0.37      0.49      0.42      5393
           4       0.42      0.54      0.47      4902
           5       0.36      0.23      0.28      1928
           6       0.37      0.17      0.23       618
           7       0.42      0.16      0.24       195

    accuracy                           0.37     20144
   macro avg       0.37      0.28      0.30     20144
weighted avg       0.36      0.37      0.36     20144

[[ 107  399  163  220   57    2    2    0]
 [ 113  874  568  984  230    7    1    0]
 [  47  624  656 1476  554   22    2    0]
 [  32  497  668 2634 1468   85    9    0]
 [   5  116  239 1533 2633  344   27    5]
 [   1   10 

/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Name: CatBoost - F1: 0.4010 - Balanced Accuracy: 0.2413 - Accuracy: 0.4010 - MCC: 0.2312 - RMSE 1.1309 - Elapsed: 247.02s
              precision    recall  f1-score   support

           0       0.49      0.03      0.05       950
           1       0.39      0.31      0.34      2777
           2       0.31      0.14      0.20      3381
           3       0.38      0.63      0.47      5393
           4       0.45      0.61      0.52      4902
           5       0.41      0.18      0.25      1928
           6       0.35      0.05      0.08       618
           7       0.00      0.00      0.00       195

    accuracy                           0.40     20144
   macro avg       0.35      0.24      0.24     20144
weighted avg       0.39      0.40      0.36     20144

[[  25  494  148  250   31    2    0    0]
 [  18  850  490 1276  141    2    0    0]
 [   6  483  485 2012  392    3    0    0]
 [   1  268  350 3381 1369   23    1    0]
 [   0   59   85 1688 2968   99    3    0]
 [   0    5 

/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [11]:
# Ordenando os resultados por MCC
df_results.sort_values(by="MCC", ascending=False)

,Model,F1,Balanced Accuracy,Accuracy,MCC,RMSE,Total Time,Confusion Matrix,Classification Report
3,XGBoost,0.39520,0.31574,0.39520,0.23961,1.14543,990.67277,[[ 172 438 177 129 32 1 1 0]\n [...,precision recall f1-score ...
11,CatBoost,0.40101,0.24130,0.40101,0.23125,1.13088,247.02237,[[ 25 494 148 250 31 2 0 0]\n [...,precision recall f1-score ...
2,RandomForest,0.38686,0.28243,0.38686,0.22038,1.17305,6.66974,[[ 93 455 132 221 47 1 1 0]\n [...,precision recall f1-score ...
10,ExtraTrees,0.37162,0.27619,0.37162,0.20271,1.21256,7.37227,[[ 107 399 163 220 57 2 2 0]\n [...,precision recall f1-score ...
7,LinearSVC,0.30729,0.35755,0.30729,0.18641,1.51554,10.17618,[[ 579 247 26 45 33 11 6 3]\n [...,precision recall f1-score ...
4,MLP,0.35107,0.25387,0.35107,0.18091,1.25908,111.96132,[[ 52 563 48 251 32 2 2 0]\n [...,precision recall f1-score ...
9,DecisionTree,0.29031,0.26230,0.29031,0.12671,1.48359,2.99482,[[ 199 281 220 157 84 6 3 0]\n [...,precision recall f1-score ...
6,NaiveBayes,0.27194,0.27550,0.27194,0.12105,1.68405,1.75968,[[ 307 275 63 116 133 32 20 4]\n [...,precision recall f1-score ...
1,LogisticRegression,0.17405,0.26198,0.17405,0.06352,2.20976,10.94814,[[ 446 156 45 72 89 90 43 9]\n [...,precision recall f1-score ...
8,KNN,0.25228,0.16532,0.25228,0.05991,1.67761,5.80867,[[ 84 269 195 265 126 10 1 0]\n [...,precision recall f1-score ...


# Add cleanlab

In [12]:
model = XGBClassifier(random_state=314, n_jobs=-1, class_weight="balanced")
preds_model = cross_val_predict(
    estimator=model,
    X=X,
    y=y,
    cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=314),
    method="predict_proba",
    n_jobs=2,
)

/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:25:12] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:25:12] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:35:32] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:35:39] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:46:02] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:46:05] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/jacob/miniforge3/envs/pape

In [13]:
lab = Datalab(data=df, label_name="categoria_renda")
lab.find_issues(features=X, pred_probs=preds_model)
lab.report()

Finding null issues ...
Error in null: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
Finding label issues ...
Finding outlier issues ...
Fitting OOD estimator based on provided features ...
Finding near_duplicate issues ...
Finding non_iid issues ...
Finding class_imbalance issues ...
Finding underperforming_group issues ...
Failed to check for these issue types: [NullIssueManager]

Audit complete. 11509 issues found in the dataset.
Here is a summary of the different kinds of issues found in the data:

     issue_type  num_issues
          label        9647
        outlier        1656
class_imbalance         195
 near_duplicate          10
        non_iid           1

Dataset Information: num_examples: 20144, num_classes: 8


----------------------- label issues -----------------------

About this issue:
	Examples whose given label is estimated to be potentially incorrect
    (e.

/home/jacob/miniforge3/envs/paper_gender_meat/lib/python3.11/site-packages/sklearn/neighbors/_base.py:246: EfficiencyWarning: Precomputed sparse input was not sorted by row values. Use the function sklearn.neighbors.sort_graph_by_row_values to sort the input by row values, with warn_when_not_sorted=False to remove this warning.
  warnings.warn(


In [14]:
lab.get_issues()

,is_label_issue,label_score,is_outlier_issue,outlier_score,is_near_duplicate_issue,near_duplicate_score,is_non_iid_issue,non_iid_score,is_class_imbalance_issue,class_imbalance_score,is_underperforming_group_issue,underperforming_group_score
0,False,0.45661,False,0.22477,False,0.81699,False,0.85173,False,1.00000,False,1.00000
1,True,0.29302,False,0.50849,False,0.51249,False,0.93128,False,1.00000,False,1.00000
2,True,0.09968,False,0.49093,False,0.54589,False,0.86444,False,1.00000,False,1.00000
3,True,0.09793,False,0.50067,False,0.50100,False,0.91735,False,1.00000,False,1.00000
4,True,0.13237,False,0.44093,False,0.60584,False,0.94196,False,1.00000,False,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...
20139,False,0.47075,False,0.56442,False,0.40316,False,0.86485,False,1.00000,False,1.00000
20140,False,0.17228,False,0.48970,False,0.49577,False,0.90693,True,0.00968,False,1.00000
20141,False,0.55776,False,0.06388,False,0.95088,False,0.84495,True,0.00968,False,1.00000
20142,True,0.08516,False,0.43670,False,0.56205,False,0.91325,False,1.00000,False,1.00000


In [15]:
label_issues = lab.get_issues()
label_issues = label_issues[["is_label_issue", "label_score"]]
label_issues

,is_label_issue,label_score
0,False,0.45661
1,True,0.29302
2,True,0.09968
3,True,0.09793
4,True,0.13237
...,...,...
20139,False,0.47075
20140,False,0.17228
20141,False,0.55776
20142,True,0.08516


In [16]:
label_issues["pred_model"] = np.argmax(preds_model, axis=1)
label_issues["original_label"] = y

label_issues

/tmp/ipykernel_468417/707172114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_issues["pred_model"] = np.argmax(preds_model, axis=1)
/tmp/ipykernel_468417/707172114.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_issues["original_label"] = y


,is_label_issue,label_score,pred_model,original_label
0,False,0.45661,4,4
1,True,0.29302,5,4
2,True,0.09968,2,4
3,True,0.09793,2,1
4,True,0.13237,4,3
...,...,...,...,...
20139,False,0.47075,6,6
20140,False,0.17228,6,7
20141,False,0.55776,7,7
20142,True,0.08516,3,2


In [17]:
# check if the difference between pred_model and original_label is greater than 1

label_issues["absolute_diff"] = (
    label_issues["pred_model"] - label_issues["original_label"]
)
label_issues["absolute_diff"] = label_issues["absolute_diff"].abs()
label_issues["is_label_issue_over_2_classes"] = label_issues["absolute_diff"] > 1
label_issues["is_label_issue_over_2_classes"].value_counts()

/tmp/ipykernel_468417/747052940.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_issues["absolute_diff"] = (
/tmp/ipykernel_468417/747052940.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_issues["absolute_diff"] = label_issues["absolute_diff"].abs()
/tmp/ipykernel_468417/747052940.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

is_label_issue_over_2_classes
False    16754
True      3390
Name: count, dtype: int64

In [18]:
label_issues

,is_label_issue,label_score,pred_model,original_label,absolute_diff,is_label_issue_over_2_classes
0,False,0.45661,4,4,0,False
1,True,0.29302,5,4,1,False
2,True,0.09968,2,4,2,True
3,True,0.09793,2,1,1,False
4,True,0.13237,4,3,1,False
...,...,...,...,...,...,...
20139,False,0.47075,6,6,0,False
20140,False,0.17228,6,7,1,False
20141,False,0.55776,7,7,0,False
20142,True,0.08516,3,2,1,False


In [19]:
label_issues.to_parquet("output/label_issues_8c.parquet")

In [20]:
label_issues.query("is_label_issue == True").groupby(
    "absolute_diff"
).label_score.describe()

,count,mean,std,min,25%,50%,75%,max
absolute_diff,,,,,,,,
1,7000.00000,0.20099,0.09055,0.00052,0.13446,0.20458,0.26668,0.46304
2,2088.00000,0.10895,0.07367,0.00016,0.04726,0.09987,0.16040,0.35823
3,481.00000,0.04871,0.04651,0.00003,0.01289,0.03474,0.07165,0.23671
4,65.00000,0.01342,0.01947,0.00011,0.00272,0.00759,0.01350,0.10995
5,8.00000,0.00440,0.00549,0.00011,0.00165,0.00266,0.00480,0.01726
6,4.00000,0.00129,0.00240,0.00001,0.00006,0.00013,0.00137,0.00489
7,1.00000,0.00009,NaN,0.00009,0.00009,0.00009,0.00009,0.00009


In [21]:
df.rename(columns={"categoria_renda": "categoria_renda_original"}, inplace=True)

In [22]:
label_issues_new = label_issues[
    [
        "label_score",
        "pred_model",
        "original_label",
        "absolute_diff",
        "is_label_issue_over_2_classes",
    ]
]

# Merging label issues to the non one-hot-encoded dataset

In [23]:
df2 = pd.read_parquet("output/data_8_classes.parquet")
df2

,idfam,UF____Unidade_da_Federacao,TIPO_SITUACAO_REG____Situacao_do_Domicilio,V0306____Condicao_na_Unidade_de_Consumo_Identifica_o_grau_de_parentesco_ou_a_natureza_da_subordinacao_existente_entre_o_morador_e_a_pessoa_de_referencia_da_sua_unidade_de_consumo,V0405____Cor_ou_raca,V0406_____tem_plano_ou_segurosaude,V0407_____teve_algum_tipo_de_rendimento_ou_executou_algum_trabalho_remunerado_ou_nao_no_periodo_de_referencia_de_12_meses,V0409____Quantos_cartoes_de_credito_tem,V0410____Quantas_contas_correntes_tem,V0414_____sabe_ler_e_escrever,ANOS_ESTUDO____Anos_de_estudo_da_pessoa_Variavel_derivada_construida_a_partir_dos_quesitos_referentes_a_educacao,V0201____Este_domicilio_e_do_tipo,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio,V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio,V0205____Quantos_comodos_tem_este_domicilio,V0206____Quantos_comodos_estao_servindo_permanentemente_de_dormitorio_para_os_moradores_deste_domicilio,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio,V0209____A_agua_utilizada_neste_domicilio_chega,V02111____Quantos_banheiros_com_chuveiro_e_vaso_sanitario_de_uso_exclusivo_dos_moradores_existem_neste_domicilio_inclusive_os_localizados_no_terreno_ou_na_propriedade,V0212____De_que_forma_e_feito_o_escoadouro_destes_banheiros_sanitarios_ou_buracos_para_dejecoes,V0213____Qual_o_principal_destino_dado_ao_lixo,V02141____A_energia_eletrica_utilizada_neste_domicilio_e_proveniente_de_rede_geral,V02161____Usa_gas_de_botijao_para_preparar_alimentos,V02162____Lenha_ou_carvao_e_utilizado_neste_domicilio_na_preparacao_dos_alimentos,V02163____Energia_eletrica_e_utilizada_neste_domicilio_na_preparacao_dos_alimentos,V02164____Outro_combustivel_oleo_querosene_etc_e_utilizado_neste_domicilio_na_preparacao_dos_alimentos,V0217____Este_domicilio_e,V0220____Existe_pavimentacao_na_rua_onde_se_localiza_este_domicilio,V0221____O_servico_de_distribuicao_dos_correios_para_este_domicilio_e_realizado,pessoas_na_familia,imc_medio_familia,energia_kcal_medio_familia,anos_estudo_medio_familia,idade_media_familia,pior_situacao_san,pessoa_referencia_mulher,RENDA_TOTAL,renda_total_sm,categoria_renda,uid
0,110000016021,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,0,1,Sim,5,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,Cimento,6,2,Rede geral de distribuição,Canalizada em pelo menos um cômodo,2,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Sim,Não,Próprio de algum morador – ainda pagando,Sim,Por entrega domiciliária [pelo carteiro],3,25,1321,7,55,SAN,Não,3855.34000,4.11456,4,0
1,110000016071,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,4,1,Sim,12,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"Cerâmica, lajota ou pedra",8,3,Rede geral de distribuição,Canalizada em pelo menos um cômodo,2,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Sim,Não,Próprio de algum morador – já pago,Sim,Por entrega domiciliária [pelo carteiro],4,23,2070,11,27,SAN,Não,6253.24000,6.67368,4,1
2,110000016121,Rondônia,Urbano,Cônjuge ou companheiro[a] de sexo diferente,Parda,Não,Sim,1,0,Sim,11,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"Cerâmica, lajota ou pedra",6,3,Poço profundo ou artesiano,Canalizada em pelo menos um cômodo,1,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Não,Não,Alugado,Sim,Por entrega domiciliária [pelo carteiro],2,25,1946,8,23,SAN,Não,4353.51000,4.64622,4,2
3,110000016131,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,1,0,Sim,10,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"Cerâmica, lajota ou pedra",7,1,Rede geral de distribuição,Canalizada em pelo menos um cômodo,2,Fossa não ligada à rede,Coletado diretamente po

In [24]:
df2 = df2.merge(label_issues_new, left_index=True, right_index=True)
df2

,idfam,UF____Unidade_da_Federacao,TIPO_SITUACAO_REG____Situacao_do_Domicilio,V0306____Condicao_na_Unidade_de_Consumo_Identifica_o_grau_de_parentesco_ou_a_natureza_da_subordinacao_existente_entre_o_morador_e_a_pessoa_de_referencia_da_sua_unidade_de_consumo,V0405____Cor_ou_raca,V0406_____tem_plano_ou_segurosaude,V0407_____teve_algum_tipo_de_rendimento_ou_executou_algum_trabalho_remunerado_ou_nao_no_periodo_de_referencia_de_12_meses,V0409____Quantos_cartoes_de_credito_tem,V0410____Quantas_contas_correntes_tem,V0414_____sabe_ler_e_escrever,ANOS_ESTUDO____Anos_de_estudo_da_pessoa_Variavel_derivada_construida_a_partir_dos_quesitos_referentes_a_educacao,V0201____Este_domicilio_e_do_tipo,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio,V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio,V0205____Quantos_comodos_tem_este_domicilio,V0206____Quantos_comodos_estao_servindo_permanentemente_de_dormitorio_para_os_moradores_deste_domicilio,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio,V0209____A_agua_utilizada_neste_domicilio_chega,V02111____Quantos_banheiros_com_chuveiro_e_vaso_sanitario_de_uso_exclusivo_dos_moradores_existem_neste_domicilio_inclusive_os_localizados_no_terreno_ou_na_propriedade,V0212____De_que_forma_e_feito_o_escoadouro_destes_banheiros_sanitarios_ou_buracos_para_dejecoes,V0213____Qual_o_principal_destino_dado_ao_lixo,V02141____A_energia_eletrica_utilizada_neste_domicilio_e_proveniente_de_rede_geral,V02161____Usa_gas_de_botijao_para_preparar_alimentos,V02162____Lenha_ou_carvao_e_utilizado_neste_domicilio_na_preparacao_dos_alimentos,V02163____Energia_eletrica_e_utilizada_neste_domicilio_na_preparacao_dos_alimentos,V02164____Outro_combustivel_oleo_querosene_etc_e_utilizado_neste_domicilio_na_preparacao_dos_alimentos,V0217____Este_domicilio_e,V0220____Existe_pavimentacao_na_rua_onde_se_localiza_este_domicilio,V0221____O_servico_de_distribuicao_dos_correios_para_este_domicilio_e_realizado,pessoas_na_familia,imc_medio_familia,energia_kcal_medio_familia,anos_estudo_medio_familia,idade_media_familia,pior_situacao_san,pessoa_referencia_mulher,RENDA_TOTAL,renda_total_sm,categoria_renda,uid,label_score,pred_model,original_label,absolute_diff,is_label_issue_over_2_classes
0,110000016021,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,0,1,Sim,5,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,Cimento,6,2,Rede geral de distribuição,Canalizada em pelo menos um cômodo,2,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Sim,Não,Próprio de algum morador – ainda pagando,Sim,Por entrega domiciliária [pelo carteiro],3,25,1321,7,55,SAN,Não,3855.34000,4.11456,4,0,0.45661,4,4,0,False
1,110000016071,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,4,1,Sim,12,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"Cerâmica, lajota ou pedra",8,3,Rede geral de distribuição,Canalizada em pelo menos um cômodo,2,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Sim,Não,Próprio de algum morador – já pago,Sim,Por entrega domiciliária [pelo carteiro],4,23,2070,11,27,SAN,Não,6253.24000,6.67368,4,1,0.29302,5,4,1,False
2,110000016121,Rondônia,Urbano,Cônjuge ou companheiro[a] de sexo diferente,Parda,Não,Sim,1,0,Sim,11,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"Cerâmica, lajota ou pedra",6,3,Poço profundo ou artesiano,Canalizada em pelo menos um cômodo,1,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Não,Não,Alugado,Sim,Por entrega domiciliária [pelo carteiro],2,25,1946,8,23,SAN,Não,4353.51000,4.64622,4,2,0.09968,2,4,2,True
3,110000016131,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,1,0,Sim,10,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"C

In [25]:
df2.rename(columns={"pred_model": "categoria_renda_corrigida"}, inplace=True)
df2.drop(columns=["original_label"], inplace=True)
df2

,idfam,UF____Unidade_da_Federacao,TIPO_SITUACAO_REG____Situacao_do_Domicilio,V0306____Condicao_na_Unidade_de_Consumo_Identifica_o_grau_de_parentesco_ou_a_natureza_da_subordinacao_existente_entre_o_morador_e_a_pessoa_de_referencia_da_sua_unidade_de_consumo,V0405____Cor_ou_raca,V0406_____tem_plano_ou_segurosaude,V0407_____teve_algum_tipo_de_rendimento_ou_executou_algum_trabalho_remunerado_ou_nao_no_periodo_de_referencia_de_12_meses,V0409____Quantos_cartoes_de_credito_tem,V0410____Quantas_contas_correntes_tem,V0414_____sabe_ler_e_escrever,ANOS_ESTUDO____Anos_de_estudo_da_pessoa_Variavel_derivada_construida_a_partir_dos_quesitos_referentes_a_educacao,V0201____Este_domicilio_e_do_tipo,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio,V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio,V0205____Quantos_comodos_tem_este_domicilio,V0206____Quantos_comodos_estao_servindo_permanentemente_de_dormitorio_para_os_moradores_deste_domicilio,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio,V0209____A_agua_utilizada_neste_domicilio_chega,V02111____Quantos_banheiros_com_chuveiro_e_vaso_sanitario_de_uso_exclusivo_dos_moradores_existem_neste_domicilio_inclusive_os_localizados_no_terreno_ou_na_propriedade,V0212____De_que_forma_e_feito_o_escoadouro_destes_banheiros_sanitarios_ou_buracos_para_dejecoes,V0213____Qual_o_principal_destino_dado_ao_lixo,V02141____A_energia_eletrica_utilizada_neste_domicilio_e_proveniente_de_rede_geral,V02161____Usa_gas_de_botijao_para_preparar_alimentos,V02162____Lenha_ou_carvao_e_utilizado_neste_domicilio_na_preparacao_dos_alimentos,V02163____Energia_eletrica_e_utilizada_neste_domicilio_na_preparacao_dos_alimentos,V02164____Outro_combustivel_oleo_querosene_etc_e_utilizado_neste_domicilio_na_preparacao_dos_alimentos,V0217____Este_domicilio_e,V0220____Existe_pavimentacao_na_rua_onde_se_localiza_este_domicilio,V0221____O_servico_de_distribuicao_dos_correios_para_este_domicilio_e_realizado,pessoas_na_familia,imc_medio_familia,energia_kcal_medio_familia,anos_estudo_medio_familia,idade_media_familia,pior_situacao_san,pessoa_referencia_mulher,RENDA_TOTAL,renda_total_sm,categoria_renda,uid,label_score,categoria_renda_corrigida,absolute_diff,is_label_issue_over_2_classes
0,110000016021,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,0,1,Sim,5,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,Cimento,6,2,Rede geral de distribuição,Canalizada em pelo menos um cômodo,2,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Sim,Não,Próprio de algum morador – ainda pagando,Sim,Por entrega domiciliária [pelo carteiro],3,25,1321,7,55,SAN,Não,3855.34000,4.11456,4,0,0.45661,4,0,False
1,110000016071,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,4,1,Sim,12,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"Cerâmica, lajota ou pedra",8,3,Rede geral de distribuição,Canalizada em pelo menos um cômodo,2,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Sim,Não,Próprio de algum morador – já pago,Sim,Por entrega domiciliária [pelo carteiro],4,23,2070,11,27,SAN,Não,6253.24000,6.67368,4,1,0.29302,5,1,False
2,110000016121,Rondônia,Urbano,Cônjuge ou companheiro[a] de sexo diferente,Parda,Não,Sim,1,0,Sim,11,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"Cerâmica, lajota ou pedra",6,3,Poço profundo ou artesiano,Canalizada em pelo menos um cômodo,1,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Não,Não,Alugado,Sim,Por entrega domiciliária [pelo carteiro],2,25,1946,8,23,SAN,Não,4353.51000,4.64622,4,2,0.09968,2,2,True
3,110000016131,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,1,0,Sim,10,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"Cerâmic

In [26]:
df2.is_label_issue_over_2_classes.value_counts(dropna=False)

is_label_issue_over_2_classes
False    16754
True      3390
Name: count, dtype: int64

In [27]:
df2

,idfam,UF____Unidade_da_Federacao,TIPO_SITUACAO_REG____Situacao_do_Domicilio,V0306____Condicao_na_Unidade_de_Consumo_Identifica_o_grau_de_parentesco_ou_a_natureza_da_subordinacao_existente_entre_o_morador_e_a_pessoa_de_referencia_da_sua_unidade_de_consumo,V0405____Cor_ou_raca,V0406_____tem_plano_ou_segurosaude,V0407_____teve_algum_tipo_de_rendimento_ou_executou_algum_trabalho_remunerado_ou_nao_no_periodo_de_referencia_de_12_meses,V0409____Quantos_cartoes_de_credito_tem,V0410____Quantas_contas_correntes_tem,V0414_____sabe_ler_e_escrever,ANOS_ESTUDO____Anos_de_estudo_da_pessoa_Variavel_derivada_construida_a_partir_dos_quesitos_referentes_a_educacao,V0201____Este_domicilio_e_do_tipo,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio,V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio,V0205____Quantos_comodos_tem_este_domicilio,V0206____Quantos_comodos_estao_servindo_permanentemente_de_dormitorio_para_os_moradores_deste_domicilio,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio,V0209____A_agua_utilizada_neste_domicilio_chega,V02111____Quantos_banheiros_com_chuveiro_e_vaso_sanitario_de_uso_exclusivo_dos_moradores_existem_neste_domicilio_inclusive_os_localizados_no_terreno_ou_na_propriedade,V0212____De_que_forma_e_feito_o_escoadouro_destes_banheiros_sanitarios_ou_buracos_para_dejecoes,V0213____Qual_o_principal_destino_dado_ao_lixo,V02141____A_energia_eletrica_utilizada_neste_domicilio_e_proveniente_de_rede_geral,V02161____Usa_gas_de_botijao_para_preparar_alimentos,V02162____Lenha_ou_carvao_e_utilizado_neste_domicilio_na_preparacao_dos_alimentos,V02163____Energia_eletrica_e_utilizada_neste_domicilio_na_preparacao_dos_alimentos,V02164____Outro_combustivel_oleo_querosene_etc_e_utilizado_neste_domicilio_na_preparacao_dos_alimentos,V0217____Este_domicilio_e,V0220____Existe_pavimentacao_na_rua_onde_se_localiza_este_domicilio,V0221____O_servico_de_distribuicao_dos_correios_para_este_domicilio_e_realizado,pessoas_na_familia,imc_medio_familia,energia_kcal_medio_familia,anos_estudo_medio_familia,idade_media_familia,pior_situacao_san,pessoa_referencia_mulher,RENDA_TOTAL,renda_total_sm,categoria_renda,uid,label_score,categoria_renda_corrigida,absolute_diff,is_label_issue_over_2_classes
0,110000016021,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,0,1,Sim,5,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,Cimento,6,2,Rede geral de distribuição,Canalizada em pelo menos um cômodo,2,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Sim,Não,Próprio de algum morador – ainda pagando,Sim,Por entrega domiciliária [pelo carteiro],3,25,1321,7,55,SAN,Não,3855.34000,4.11456,4,0,0.45661,4,0,False
1,110000016071,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,4,1,Sim,12,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"Cerâmica, lajota ou pedra",8,3,Rede geral de distribuição,Canalizada em pelo menos um cômodo,2,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Sim,Não,Próprio de algum morador – já pago,Sim,Por entrega domiciliária [pelo carteiro],4,23,2070,11,27,SAN,Não,6253.24000,6.67368,4,1,0.29302,5,1,False
2,110000016121,Rondônia,Urbano,Cônjuge ou companheiro[a] de sexo diferente,Parda,Não,Sim,1,0,Sim,11,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"Cerâmica, lajota ou pedra",6,3,Poço profundo ou artesiano,Canalizada em pelo menos um cômodo,1,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Não,Não,Alugado,Sim,Por entrega domiciliária [pelo carteiro],2,25,1946,8,23,SAN,Não,4353.51000,4.64622,4,2,0.09968,2,2,True
3,110000016131,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,1,0,Sim,10,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"Cerâmic

In [28]:
df2.rename(
    columns={"categoria_renda_corrigida": "categoria_renda_automatizada"}, inplace=True
)

In [29]:
df2["categoria_renda_corrigida"] = df2.apply(
    lambda x: (
        x["categoria_renda_automatizada"]
        if x["is_label_issue_over_2_classes"] == True
        else x["categoria_renda"]
    ),
    axis=1,
)

In [30]:
df2

,idfam,UF____Unidade_da_Federacao,TIPO_SITUACAO_REG____Situacao_do_Domicilio,V0306____Condicao_na_Unidade_de_Consumo_Identifica_o_grau_de_parentesco_ou_a_natureza_da_subordinacao_existente_entre_o_morador_e_a_pessoa_de_referencia_da_sua_unidade_de_consumo,V0405____Cor_ou_raca,V0406_____tem_plano_ou_segurosaude,V0407_____teve_algum_tipo_de_rendimento_ou_executou_algum_trabalho_remunerado_ou_nao_no_periodo_de_referencia_de_12_meses,V0409____Quantos_cartoes_de_credito_tem,V0410____Quantas_contas_correntes_tem,V0414_____sabe_ler_e_escrever,ANOS_ESTUDO____Anos_de_estudo_da_pessoa_Variavel_derivada_construida_a_partir_dos_quesitos_referentes_a_educacao,V0201____Este_domicilio_e_do_tipo,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio,V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio,V0205____Quantos_comodos_tem_este_domicilio,V0206____Quantos_comodos_estao_servindo_permanentemente_de_dormitorio_para_os_moradores_deste_domicilio,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio,V0209____A_agua_utilizada_neste_domicilio_chega,V02111____Quantos_banheiros_com_chuveiro_e_vaso_sanitario_de_uso_exclusivo_dos_moradores_existem_neste_domicilio_inclusive_os_localizados_no_terreno_ou_na_propriedade,V0212____De_que_forma_e_feito_o_escoadouro_destes_banheiros_sanitarios_ou_buracos_para_dejecoes,V0213____Qual_o_principal_destino_dado_ao_lixo,V02141____A_energia_eletrica_utilizada_neste_domicilio_e_proveniente_de_rede_geral,V02161____Usa_gas_de_botijao_para_preparar_alimentos,V02162____Lenha_ou_carvao_e_utilizado_neste_domicilio_na_preparacao_dos_alimentos,V02163____Energia_eletrica_e_utilizada_neste_domicilio_na_preparacao_dos_alimentos,V02164____Outro_combustivel_oleo_querosene_etc_e_utilizado_neste_domicilio_na_preparacao_dos_alimentos,V0217____Este_domicilio_e,V0220____Existe_pavimentacao_na_rua_onde_se_localiza_este_domicilio,V0221____O_servico_de_distribuicao_dos_correios_para_este_domicilio_e_realizado,pessoas_na_familia,imc_medio_familia,energia_kcal_medio_familia,anos_estudo_medio_familia,idade_media_familia,pior_situacao_san,pessoa_referencia_mulher,RENDA_TOTAL,renda_total_sm,categoria_renda,uid,label_score,categoria_renda_automatizada,absolute_diff,is_label_issue_over_2_classes,categoria_renda_corrigida
0,110000016021,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,0,1,Sim,5,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,Cimento,6,2,Rede geral de distribuição,Canalizada em pelo menos um cômodo,2,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Sim,Não,Próprio de algum morador – ainda pagando,Sim,Por entrega domiciliária [pelo carteiro],3,25,1321,7,55,SAN,Não,3855.34000,4.11456,4,0,0.45661,4,0,False,4
1,110000016071,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,4,1,Sim,12,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"Cerâmica, lajota ou pedra",8,3,Rede geral de distribuição,Canalizada em pelo menos um cômodo,2,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Sim,Não,Próprio de algum morador – já pago,Sim,Por entrega domiciliária [pelo carteiro],4,23,2070,11,27,SAN,Não,6253.24000,6.67368,4,1,0.29302,5,1,False,4
2,110000016121,Rondônia,Urbano,Cônjuge ou companheiro[a] de sexo diferente,Parda,Não,Sim,1,0,Sim,11,Casa,Alvenaria com revestimento / Taipa com revesti...,Telha sem laje de concreto,"Cerâmica, lajota ou pedra",6,3,Poço profundo ou artesiano,Canalizada em pelo menos um cômodo,1,Fossa não ligada à rede,Coletado diretamente por serviço de limpeza,Sim,Sim,Não,Não,Não,Alugado,Sim,Por entrega domiciliária [pelo carteiro],2,25,1946,8,23,SAN,Não,4353.51000,4.64622,4,2,0.09968,2,2,True,2
3,110000016131,Rondônia,Urbano,Pessoa de referência da UC,Parda,Não,Sim,1,0,Sim,10,Casa,Alvenaria com revestimento / Taipa com revesti...,

In [31]:
df2.to_parquet("output/data_8_classes_not_onehot_income_fixed.parquet")

In [32]:
df["categoria_renda_corrigida"] = df2["categoria_renda_corrigida"].copy()

In [33]:
df

,idfam,V0409____Quantos_cartoes_de_credito_tem,V0410____Quantas_contas_correntes_tem,ANOS_ESTUDO____Anos_de_estudo_da_pessoa_Variavel_derivada_construida_a_partir_dos_quesitos_referentes_a_educacao,V0205____Quantos_comodos_tem_este_domicilio,V0206____Quantos_comodos_estao_servindo_permanentemente_de_dormitorio_para_os_moradores_deste_domicilio,V02111____Quantos_banheiros_com_chuveiro_e_vaso_sanitario_de_uso_exclusivo_dos_moradores_existem_neste_domicilio_inclusive_os_localizados_no_terreno_ou_na_propriedade,pessoas_na_familia,imc_medio_familia,energia_kcal_medio_familia,anos_estudo_medio_familia,idade_media_familia,RENDA_TOTAL,renda_total_sm,categoria_renda_original,uid,TIPO_SITUACAO_REG____Situacao_do_Domicilio_Urbano,V0405____Cor_ou_raca_Amarela,V0405____Cor_ou_raca_Branca,V0405____Cor_ou_raca_Indígena,V0405____Cor_ou_raca_Parda,V0405____Cor_ou_raca_Preta,V0405____Cor_ou_raca_Sem declaração,V0406_____tem_plano_ou_segurosaude_Sim,V0407_____teve_algum_tipo_de_rendimento_ou_executou_algum_trabalho_remunerado_ou_nao_no_periodo_de_referencia_de_12_meses_Sim,V0414_____sabe_ler_e_escrever_Sim,V0201____Este_domicilio_e_do_tipo_Apartamento,V0201____Este_domicilio_e_do_tipo_Casa,"V0201____Este_domicilio_e_do_tipo_Habitação em casa de cômodos, cortiço ou cabeça de porco",V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Alvenaria com revestimento / Taipa com revestimento,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Alvenaria sem revestimento,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Madeira apropriada para construção [aparelhada],V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Madeira aproveitada,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Outro material,V0202____Qual_e_o_material_que_predomina_na_construcao_das_paredes_externas_deste_domicilio_Taipa sem revestimento,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Madeira apropriada para construção [aparelhada],V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Outro material,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Somente laje de concreto,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Telha com laje de concreto,V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Telha sem laje de concreto,"V0203____Qual_e_o_material_que_predomina_na_cobertura_telhado_deste_domicilio_Zinco, alumínio ou chapa metálica","V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Cerâmica, lajota ou pedra",V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Cimento,V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Madeira apropriada para construção [aparelhada],V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Outro Material,V0204____Qual_e_o_material_que_predomina_no_piso_deste_domicilio_Terra,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Fonte ou nascente,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Outra forma,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Poço profundo ou artesiano,"V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Poço raso, freático ou cacimba",V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Rede geral de distribuição,V0207____Qual_a_principal_forma_de_abastecimento_de_agua_utilizada_neste_domicilio_Água da chuva armazenada,V0209____A_agua_utilizada_neste_domicilio_chega_Canalizada em pelo menos um cômodo,V0209____A_agua_utilizada_neste_domicilio_chega_Canalizada só na propriedade ou terreno,V0209____A_agua_utilizada_neste_domicilio_chega_Não canalizada,V0212____De_que_forma_e_feito_

In [34]:
df.to_parquet("output/data_8_classes_one_hot_encoded_income_fixed.parquet")